Instalación de bibliotecas:

!pip install opencv-python
!pip install opencv-python-headless

Importar las bibliotecas necesarias:

In [7]:
import cv2
import numpy as np
import matplotlib.pyplot as plt


Leer el video usando OpenCV:

In [8]:
cap = cv2.VideoCapture('test_video.mp4')

Definir los puntos de referencia para la región de interés (ROI) en la imagen del video. La ROI es la región en la que se buscarán los carriles:

In [9]:
# Definir puntos de referencia para la región de interés
height, width = cap.read()[1].shape[:2]

#ROI_vertices = [(0, height), (width/2, height/2), (width, height)]
ROI_vertices = [(200, height), (1100,height), (550, 250)]
#ROI_vertices = [(0, 540), (200,300), (650, 300), (850, 540)]
#ROI_vertices = [(200, height), (1100,height), (550, 250), (right_bottom_x, right_bottom_y)]



In [10]:
def draw_lines(frame, lines):
    """
    Esta función toma una imagen de entrada y una lista de líneas, y dibuja las líneas en la imagen.
    """
    # Crear una imagen en negro para dibujar las líneas
    line_image = np.zeros_like(frame)

    # Dibujar cada línea en la imagen
    if lines is not None:
        for line in lines:
            x1, y1, x2, y2 = line.reshape(4)
            cv2.line(line_image, (x1, y1), (x2, y2), (255, 0, 0), 10)

    # Combinar la imagen con las líneas con la imagen de entrada utilizando una operación bitwise OR
    line_frame = cv2.addWeighted(frame, 0.8, line_image, 1, 0.0)

    return line_frame

Crear una función para procesar cada cuadro del video. La función debe realizar las siguientes tareas:
Definir la ROI utilizando los puntos de referencia definidos anteriormente.
Convertir la imagen de entrada a escala de grises.
Aplicar un filtro gaussiano para suavizar la imagen y reducir el ruido.
Utilizar el operador Canny para detectar los bordes de los carriles.
Utilizar la transformada de Hough para detectar las líneas en la imagen.
Dibujar las líneas detectadas en la imagen de salida.

In [11]:
def process_frame(frame):
    # Definir la ROI utilizando los puntos de referencia definidos anteriormente
    mask = np.zeros_like(frame)

    # Convertir los puntos de la ROI a un tipo de datos adecuado y verificar la forma del vector de entrada
    vertices = np.array([ROI_vertices], dtype=np.int32)
    #if vertices.shape != (1, 4, 2):
     #   raise ValueError('ROI vertices must have shape (1, 4, 2)')

    cv2.fillPoly(mask, vertices, (255, 255, 255))
    masked_frame = cv2.bitwise_and(frame, mask)

    # Convertir la imagen de entrada a escala de grises
    gray = cv2.cvtColor(masked_frame, cv2.COLOR_BGR2GRAY)

    # Aplicar un filtro gaussiano para reducir el ruido
    blur = cv2.GaussianBlur(gray, (5, 5), 0)

    # Detectar los bordes de la imagen utilizando el operador Canny
    edges = cv2.Canny(blur, threshold1=50, threshold2=150)

    # Definir los parámetros de la transformada de Hough
    rho = 1  # Resolución en píxeles de la distancia rho
    theta = np.pi / 180  # Resolución en radianes del ángulo theta
    threshold = 15  # Umbral mínimo de votos para una línea
    min_line_length = 40  # Longitud mínima en píxeles de una línea
    max_line_gap = 20  # Máxima brecha en píxeles entre segmentos de línea para conectarlos en una sola línea

    # Ejecutar la transformada de Hough
    lines = cv2.HoughLinesP(edges, rho, theta, threshold, np.array([]), min_line_length, max_line_gap)

    # Dibujar las líneas detectadas en la imagen de salida
    line_frame = draw_lines(frame, lines)

    return line_frame



Procesar cada cuadro del video y mostrar la imagen de salida:

In [12]:
# Procesar cada cuadro del video y mostrar la imagen de salida
while(cap.isOpened()):
    ret, frame = cap.read()

    if ret == True:
        line_frame = process_frame(frame)
        cv2.imshow('Video', line_frame)

        # Salir al presionar la tecla 'q'
        if cv2.waitKey(1) &  0xFF == ord('q'):
            cap.release()
            cv2.destroyAllWindows()
            break
    else:
        cap.release()
        cv2.destroyAllWindows()
        break

